## Gemeinsame Toolbox für Topic Modelling, PyDelta etc.

### Ausgangssituation

* Der TMW-Workflow sieht eine statische Schritt-für-Schritt-Aufbereitung des Corpus vor: Nach jedem Verarbeitungsschritt werden die Daten in eine Dateistruktur geschrieben bzw. daraus gelesen, die ein dem Schritt entsprechendes Format hat. Dateiiteration etc. werden in jeder der recht großen Funktionen repliziert.
* PyDelta interessiert größtenteils nur für die Darstellung als Featurematrix. Zum Aufbau der Featurematrix gibt es eine Featureextraktionsklasse, in der die einzelnen Schritte zwar (durch Optionen oder Subclassing) konfigurierbar sind, bei der jedoch immer eine Featurematrix herauskommt und keine Repräsentation der einzelnen tokenisierten Texte erfolgt
* Tokenisierung etc. erfolgt nach unterschiedlichen Techniken: NLTK, Reguläre Ausdrücke, Auslesen aus tokenisiertem Text (DARIAH-Wrapper, angedacht)

Für eine möglichst breit nutzbare Library ist es sinnvoll, jede einzelne Funktionen möglichst genau eine Aufgabe erledigen zu lassen – und möglichst wenige globale Annahmen zu treffen. Eine Segmentierfunktion, die z.B. Texte in Stücke zu je $n$ Tokens zerlegt (wird z.B. in TMW oder in Andreas' Zeta-Implementierung genutzt), sollte sich genau auf das Segmentieren beschränken: Sie bekommt als Eingabe eine Sequenz von Tokens und gibt als Ergebnis eine Sequenz von Sequenzen von Tokens zurück. Sie übernimmt weder das Tokenisieren noch das Schreiben von Dateien – dies sind projektspezifische Funktionen, die von der Library durch Hilfsfunktionen unterstützt werden können, aber nicht müssen.

Dieses Prinzip erlaubt es, die eigentliche Nutzfunktion in ganz unterschiedlichen Kontexten zu nutzen, z. B. mit bereits vortokenisierten Texten, die im Speicher gehalten werden, ebenso wie mit Texten von Platte.

### Grundprinzipien

Mein Vorschlag ist, in der Toolbox so ca. drei Gruppen von Funktionen aufzunehmen:

#### Basisfunktionen: Funktionen, die irgendwas mit Texten / Featureströmen machen

Das betrifft zum Beispiel:

* Tokenisieren
* n-Gramme produzieren
* Segmentieren
* Durch Treetagger laufen lassen
* Features zählen, also von Featurestrom zu Häufigkeitstabelle (öh naja, das ist eigentlich `collections.Counter(sequence)` …)

Diese Funktionen sollten nach Möglichkeit _keine_ Dateioperationen beinhalten, sondern einfach als Eingabe z.B. einen String oder eine Sequenz von Strings und als Ausgabe z.B. eine Sequenz von Tokens oder eine Sequenz von Sequenzen von Tokens (Segmentieren) liefern. 

Diese Funktionen liefern die Basisfunktionalität, die für alle Projekte, egal wie die die Daten verwalten, zur Verfügung steht und unabhängig benutzt werden kann.

#### Lesen (und ggf. Schreiben) von Einzeldateien

Das sind Funktionen, die ggf. konfigurierbar bestimmte von uns unterstützte Dateien einlesen (z.B. DKPro-Wrapper, TEI, TreeTagger-Format?) und Daten liefern, die an die Basisfunktionen weitergeleitet werden können. Also z.B. Textinhalte, Tokenströme usw. Die Eingabe ist ein Dateipfad / Datei plus weitere Optionen (z.B. XPath-Filter), die Ausgabe abhängig von der Ausgabe z.B. eine Sequenz von Tokens (oder sonstigen Features)

#### Verwaltung von Metadaten, Sequenzen und Dateinamen

Metadaten werden intern einer Tabelle (Dataframe) verwaltet, die als Index IDs und Metadaten in den Spalten verwendet. Wir bieten Hilfsfunktionen an, die z.B. solche Tabellen aufbauen, aus Metadaten auf Dateinamen mappen, nach Metadatenwerten gruppieren usw. Eine leichtgewichtige Kapselung unterstützt die Segmentierung, die IDs stehen dabei als Tupel zur Verfügung.

Bei Funktionen, die mehrere Dokumente verarbeiten, spielt die Reihenfolge eine Rolle, d.h. es werden wie z.B. bei matplotlib üblich mehrere gleichlange Listen mit korrespondierenden Daten übergeben.

#### Pipelines

Über Hilfsfunktionen (im Zusammenhang mit Metadaten) und Rezepte den Aufbau von Pipelines aus den o.g. Komponenten unterstützen.

## Funktionen zur Aufbereitung eines Dokuments / Feature-Stroms

Typische Funktionen machen irgendwas mit einem Iterable von Ausgangsdaten und liefern ein Iterable von Zieldaten. Das lässt sich oft einfach als Generator implementieren, wie das hier für einen Tokenizer gezeigt wird:

In [186]:
import regex as re
from itertools import islice, chain, tee
from functools import partial
from glob import glob
from collections.abc import Iterable
import os

def tokenize_re(lines, token=re.compile(r'(\p{L}+|\n+)')):
    """
    Tokenizes a data stream (e.g., sequence of lines) using a simple regular expression.
    
    Args:
       lines: iterable of strings to tokenize, e.g. an open text file or a list of lines
       token: compiled regular expression that identifies a token. The default accepts sequences of letters and sequences of linebreaks
    """
    for line in lines:
        yield from token.findall(line)

(eine alternative Variante, die nltk's `tokenize_sentences` entspricht, tokenisiert jedes Item in lines und liefert wiederum eine Liste von Iterables, dem könnte man eine Absatzerkennung voranstellen. 

Man kann sich nun auch leicht etwas vorstellen, das so einen Featurestrom bearbeitet. Als Beispiel eine n-Gramm-Funktion, die Wort-n-Gramme extrahiert:

In [90]:
def ngrams(iterable, n=2, sep=None):
    """
    Transforms an iterable into an iterable of ngrams.

    Args:
       iterable: Input data
       n (int): Size of each ngram
       sep (str): Separator string for the ngrams

    Yields:
       if sep is None, this yields n-tuples of the iterable. If sep is a
       string, it is used to join the tuples

    Example:
        >>> list(ngrams('This is a test'.split(), n=2, sep=' '))
        ['This is', 'is a', 'a test']
    """
    if n == 1:
        return iterable

    # Multiplicate input iterable and advance iterable n by n tokens
    ts = tee(iterable, n)
    for i, t in enumerate(ts[1:]):
        for __ in range(i + 1):
            next(t, None)

    tuples = zip(*ts)

    if sep is None:
        return tuples
    else:
        return map(sep.join, tuples)

Hier nun ein komplexeres Beispiel: Die Funktion `segmentize` segmentiert eine Featuresequenz (und berücksichtigt dabei auf Wunsch Absatzgrenzen). 

Die Funktion kümmert sich tatsächlich _nur_ um das Segmentieren, Dateioperationen sind ebenso außen vor wie das Tokenisieren. Das (Haupt-)Argument der Funktion ist eine Featuresequenz, wie sie beispielsweise aus dem Tokenizer fällt, das Ergebnis ist eine Sequenz von Segmenten, wobei jedes Segment wiederum eine Sequenz (hier sogar eine Liste) von Features ist.

In [6]:
# Hilfsfunktionen:
def first_true_index(iterable, predicate):
    for index, item in enumerate(iterable):
        if predicate(item):
            return index
    raise ValueError("No matching value found.")
        
def last_true_index(sequence, predicate):
    """
    Finds the last item in a sequence for which the given predicate is true.
    
    Args:
       sequence: a sequence of items. Must be reversible.
       predicate: a function that evaluates an item of the sequence
       
    Returns:
       the index of the last item for which the condition is true
    Raises:
       ValueError: if no item has been found
    """
    for index, item in enumerate(reversed(sequence)):
        if predicate(item):
            return len(sequence) - index
    raise ValueError("No matching value found.")
    

def segmentize(tokens, size=5000, tolerance=0, is_paragraph=re.compile('\n+').match):
    """
    Cuts the given stream of tokens into segments of a specific length, with optional support
    for trying to keep paragraphs together.
    
    Args:
        tokens (Iterable): An iterable of tokens that is to be segmented
        size (int): Target size of each segment. The last segment will probably be smaller.
        tolerance (int or float): If non-zero, adjust segment size to 
            break segments at paragraph boundaries. If this value is an integer,
            it is the actual number of tokens to look ahead or behind, if its a
            float, we look round(size*tolerance) tokens back or forth.            
        is_paragraph (Callable): A function that returns a truish value if the token
            is a paragraph boundary. Only relevant if tolerance_factor is != 0.
    Todo:
        Ggf. ist das intuitiver, wenn man die Absätze als Listen hereingibt und nicht
        mit einem Trennzeichen
    """
    tokens = iter(tokens)  # if tokens were a Sequence, islice would always index from the start
    current_segment = list(islice(tokens, size))
    fuzz = tolerance if isinstance(tolerance, int) else round(size * tolerance)
    segment_tail = size - fuzz  # start looking for paragraph boundary here
    while current_segment:
        if fuzz:
            try:
                # Find para break is in the last tokens:
                parapos = last_true_index(current_segment[segment_tail:], is_paragraph) + segment_tail
                # If found, yield segment shortened up to the paragraph break ...
                yield current_segment[:parapos]
                # ... and let the next segment start with the remainder.
                carry = current_segment[parapos:]
                current_segment = list(chain(carry, islice(tokens, size-len(carry))))
            except ValueError:
                # If this hasn't worked, look ahead if there is a paragraph marker in the next _fuzz_ tokens.
                lookahead = list(islice(tokens, fuzz))                                
                try:
                    parapos = first_true_index(lookahead, is_paragraph) + 1
                    # If so, enhance current segment ...
                    yield current_segment + lookahead[:parapos]
                    # ... and let the next segment start with the remainder.
                    carry = lookahead[parapos:]
                    current_segment = list(chain(carry, islice(tokens, size-(len(carry)))))
                except ValueError: 
                    # Otherwise, use current segment as is ...
                    yield current_segment
                    # ... and let the next segment just start with the stuff we read to look ahead.
                    current_segment = lookahead + list(islice(tokens, size-len(lookahead)))
        else:            
            yield current_segment
            current_segment = list(islice(tokens, size))            

Demo:

In [6]:
tokens = tokenize_re(["Teste mich.\nIch bin ein toller Test.\nEin ganz toller Test."])
list(segmentize(tokens, size=5, tolerance=1.0))

[['Teste', 'mich', '\n'],
 ['Ich', 'bin', 'ein', 'toller', 'Test', '\n'],
 ['Ein', 'ganz', 'toller', 'Test']]

### Manueller Umgang mit z.B. Dateien

Diese Kernfunktionen kann man nun im Grunde so benutzen, wie sie sind, und sich die eigene Dateilogik drumherum programmieren.

In [7]:
list(enumerate("abc"))

[(0, 'a'), (1, 'b'), (2, 'c')]

In [8]:
in_dir = '/home/tv/git/pydelta/corpus_DE'
out_dir = '/tmp/corpus_DE/segments'
from glob import glob
import os
import shutil

os.makedirs(out_dir, exist_ok=True)
for file in glob(os.path.join(in_dir, '*.txt')):
    id = os.path.basename(file)
    with open(file, encoding="utf-8") as f:
        for n, segment in enumerate(segmentize(tokenize_re(f), tolerance=0.1)):
            outfilename = os.path.join(out_dir, 
                "{id}§{segment:>04}.txt".format_map(dict(id=id, segment=n)))        
            with open(outfilename, mode="wt", encoding="utf-8") as out:
                out.write(" ".join(token for token in segment))

## Umgang mit Metadaten & Korpora

Nun ist es, nachdem wir aus unseren Nutzfunktionen das immergleiche Dateihandling herausgefactored haben, sinnvoll, das Dateihandling ebenso wie Nutzfunktionen als anpass- und austauschbare Komponenten anzubieten. Mein Vorschlag dazu:

* Ein eher einfaches Objekt verwaltet die Metadaten
* Das Objekt bietet u. a. Funktionen, um Dateinamen zu produzieren

Die Basisklasse dazu enthält bereits Methoden zur Konstruktion von Dateinamen, zum Handling von Segmenten etc. Das konkrete Speichern der Metadaten überlassen wir indes konkreten Implementierungen:

In [141]:
import abc

class AbstractCorpus:
    
    def __init__(self, basedir='.', default_pattern='{Index}.txt'):
        self.basedir = basedir
        self.default_pattern = default_pattern
        self.segments = None
        
    @abc.abstractmethod
    def list_docs(self):
        """
        Returns an iterable over Mappings that describe a corpus.
        
        It is expected that each Mapping has a key 'Index' with
        unique values, and that each call lists the documents in 
        the same order.
        """
    
    def list_segments(self):
        """
        Returns an iterable over Mappings that describe a segmented corpus.
        
        There is a dict for each segment with the metadata, and an int-valued
        item `segment` identifies the segment within its document.
        """
        if self.segments is None:
            raise ValueError("Not segmentized yet")
        for doc, segcount in self.list_docs(), self.segments:
            for seg in range(segcount):
                yield dict(doc, segment=seg)
        
    def filenames(self, basedir=None, pattern=None, segments=False):
        """
        Yields a filename for each entry in the metadata.
        
        
        """        
        if basedir is None:
            basedir = self.basedir
        if pattern is None:
            pattern = self.default_pattern        
        items = self.list_segments() if segments else self.list_docs()        
        for doc in items:
            filename = os.path.join(basedir, pattern.format_map(doc))
            yield filename
                        
    def forall(self, function, *args, basedir=None, pattern=None, segments=False, **kwargs):
        """
        Calls the given function for each filename.
        
        If additional positional arguments are given, they must each be an iterable
        of the same length that corresponds to the list of filenames.
        
        So this is essentially equivalent to:
        
            map(partial(function, **kwargs), self.filenames(basedir, pattern), *args)
            
        Todo:
        
            This could get two boolean arguments to enable convenience behaviour:
            
            tee: If True, call the function with a copy of the iterable. Can be
                 as intermediate pipeline step that dumps something.
            consume: If True, consume function's result and return nothing.
            
            Both options would force the pipeline to run immediately.
                 
        """        
        for args in zip(self.filenames(basedir, pattern, segments), *args):
            yield function(*args, **kwargs)
            
    def flatten_segments(self, documents):
        """
        Flatten a segment structure and record segment counts.
        
        `documents` is expected to be an Iterable of documents that matches this 
        corpus' metadata. Each document is expected to be an Iterable of segments.
        Each segment is expected to be an Iterable of features (like tokens).
        
        This method yields each segment, thus it flattens the structure by one 
        level. Additionally, it keeps track of segments by counting the number of 
        segments in each document in this object's `segments` field. Afterwards,
        you can pass `segments=True` to `filenames` and `forall`.
        """
        self.segments = []
        for document in documents:
            self.segments.append(0)
            for segment in document:
                self.segments[-1] += 1
                yield segment

Hier ist eine einfache Implementierung, die die Metadaten in einem Pandas-Dataframe verwaltet:

In [142]:
import pandas as pd

class TableCorpus(AbstractCorpus):
    
    def __init__(self, data, **kwargs):
        super().__init__(**kwargs)
        self.metadata = pd.DataFrame(data)
        
    def list_docs(self):
        return (t._asdict() for t in self.metadata.itertuples())
    
    # TODO dies könnte was zum Filtern bekommen

Falls wir diese Metadaten ad-hoc aus den Dateinamen befüllen wollen, kann uns hier eine Funktion einen entsprechenden Dataframe bauen:

In [104]:
def fn2metadata(glob_pattern='corpus/*.txt', fn_pattern=re.compile('(?<author>[^_]+)_(?<title>.+)'), index=None):
    """
    Extracts basic metadata filenames.
    
    Args:
       glob_pattern (str): A glob pattern matching the files to list, cf. glob.glob
       fn_pattern (re.Regex): A regular expression that extracts metadata fields from the files' base name. 
           The pattern must contain named groups (which have the form `(?<name>pattern)`, where name is
           the name of the metadata field and pattern is the part of the re matching this field name).
           The default will expect file names that contain a `_`, and it will assign everything before
           the _ to the `author` field and everything after to the `title` field.
       index (str): Name of the column that will be used as index column. If None, an artificial index
           (integers, starting at 0) will be used.
    Returns:
        pd.DataFrame with the following columns:
           * basename: filename without path or extension
           * filename: full filename
           * one column for every named pattern that matched for at least one file
    """
    metadata_list = []
    for filename in glob(glob_pattern):
        basename, __ = os.path.splitext(os.path.basename(filename))
        md = fn_pattern.match(basename).groupdict()
        md["basename"] = basename
        md["filename"] = filename
        metadata_list.append(md)
    metadata = pd.DataFrame(metadata_list)
    if index is not None:
        metadata = metadata.set_index(index)
    return metadata

### Lesen von Dateien

Nun kann man eine simple Funktion bauen, um einfache Textdateien zu lesen bzw. zu schreiben:

In [145]:
def read_text(filename, **kwargs):
    with open(filename, "rt", **kwargs) as f:
        return f.read()
    
def write_lines(filename, lines, **kwargs):    
    with open(filename, "wt", **kwargs) as f:
        f.writelines(lines)

Diese simplen Funktionen können jetzt mit `forall` eingesetzt werden:

In [150]:
corpus = TableCorpus(fn2metadata('/home/tv/git/pydelta.2015/test/corpus3/*.txt', index='basename'))
contents = corpus.forall(read_text, pattern='{filename}', encoding="UTF-8")
corpus.forall(write_lines, contents, pattern='/tmp/target/{Index}.out', encoding="UTF-8")

<generator object AbstractCorpus.forall at 0x7fe1e6c5ff68>

`forall`, so wie es jetzt ist, liefert einen Generator. Erst das Abarbeiten z.B. mit explizitem list führt zum Ausführen der Pipeline!

In [151]:
list(_)

[None, None, None, None, None, None, None, None, None]

### Featuretabelle

In [71]:
from collections import Counter

Wenn wir eine Sequenz z.B. von $n$-Grammen haben, können wir die leicht zählen, indem wir einfach `collections.Counter` benutzen:

In [72]:
Counter(tokenize_re(["Hallo, ich bin ein Test. Ein kleiner Test."]))

Counter({'Ein': 1,
         'Hallo': 1,
         'Test': 2,
         'bin': 1,
         'ein': 1,
         'ich': 1,
         'kleiner': 1})

Eine Frequenztabelle daraus als Dataframe baut uns die folgende Hilfsfunktion

In [105]:
# Aufbau des Corpus
md = fn2metadata('/home/tv/git/pydelta.2015/test/corpus3/*.txt')
corpus = TableCorpus(md, default_pattern="{Index:>04}.txt")

# Ein ganz simpler Tokenizer
tokenizer = re.compile('\p{L}+').findall

# Pipeline
texts = corpus.forall(read_text, pattern="{filename}") # -> Iterable von Texten
tokenized = map(tokenizer, texts)                      # -> Iterable von Iterables von Tokens
bigrams = map(partial(ngrams, n=2, sep=' '), tokenized)# -> Iterable von Iterables aus Bigrammen
counters = map(Counter, bigrams)                       # -> Iterable von Countern

# Jetzt zusammenfügen
absfreqs = pd.DataFrame(list(counters),     index=corpus.metadata.index).fillna(0).T
                        # Iterable nich ok  # Metadaten 

,author,basename,filename,title
0,"Fontane,-Theodor","Fontane,-Theodor_Der-Stechlin",/home/tv/git/pydelta.2015/test/corpus3/Fontane...,Der-Stechlin
1,"Fontane,-Theodor","Fontane,-Theodor_Effi-Briest",/home/tv/git/pydelta.2015/test/corpus3/Fontane...,Effi-Briest
2,"Fontane,-Theodor","Fontane,-Theodor_Frau-Jenny-Treibel",/home/tv/git/pydelta.2015/test/corpus3/Fontane...,Frau-Jenny-Treibel
3,"Marlitt,-Eugenie","Marlitt,-Eugenie_Das-Geheimnis-der-alten-Mamsell",/home/tv/git/pydelta.2015/test/corpus3/Marlitt...,Das-Geheimnis-der-alten-Mamsell
4,"Marlitt,-Eugenie","Marlitt,-Eugenie_Das-Heideprinzesschen",/home/tv/git/pydelta.2015/test/corpus3/Marlitt...,Das-Heideprinzesschen
5,"Marlitt,-Eugenie","Marlitt,-Eugenie_Die-Frau-mit-den-Karfunkelste...",/home/tv/git/pydelta.2015/test/corpus3/Marlitt...,Die-Frau-mit-den-Karfunkelsteinen
6,"Raabe,-Wilhelm","Raabe,-Wilhelm_Die-Chronik-der-Sperlingsgasse","/home/tv/git/pydelta.2015/test/corpus3/Raabe,-...",Die-Chronik-der-Sperlingsgasse
7,"Raabe,-Wilhelm","Raabe,-Wilhelm_Im-alten-Eisen","/home/tv/git/pydelta.2015/test/corpus3/Raabe,-...",Im-alten-Eisen
8,"Raabe,-Wilhelm","Raabe,-Wilhelm_Stopfkuchen","/home/tv/git/pydelta.2015/test/corpus3/Raabe,-...",Stopfkuchen


## Pipeline

Häufig wollen wir zahlreiche einfache Operationen nacheinander auf unseren Daten ausführen. Eine einfache Pipeline-Funktion kann uns dabei helfen:

In [152]:
def pipeline(*functions, start=None):
    data = start
    for function in functions:
        if data is None:
            data = function()
        elif isinstance(data, Iterable):
            data = map(function, data)
        else:
            data = function(data)
    return data

Diese Funktion bekommt eine Reihe von Funktionen $f_1, f_2, f_3, …$ übergeben. Jede Funktion wird dabei auf dem Ergebnis der Vorherigen ausgeführt. Als Convenience ist allerdings noch eine Iteration eingebaut, falls die Daten iterierbar sind. Damit können wir eine Pipeline über mehrere Stufen schreiben.

Im folgenden mal ein Beispiel. Wir haben unser `corpus`. Wir wollen nun:

1. Die Dateien mit ihren Originaldateinamen (Spalte `filename`) einlesen
2. Groß-/Kleinschreibung in jeder Datei ignorieren
3. jede Datei tokenisieren
4. Bigramme bilden, die mit einem `' '` getrennt sind
5. die Bigramme zählen
6. eine Frequenztabelle erstellen.

Die ersten fünf Schritte können wir mit unserer Pipeline-Funktion gleich hintereinanderschreiben.

Noch ein Hinweis zur funktionalen Schreibweise: Jeder Schritt in der Pipeline ist eine Funktion, die _genau ein_ Argument bekommt. Wir rufen die Funktion hier nicht auf, sondern übergeben sie als Objekt, d.h. die Aufrufklammern fehlen.

Beispiel unser Tokenizer:

In [155]:
tokenizer = re.compile("\w+").findall    # hier keine Aufrufklammern
tokenizer("Ich bin ein Test.")           # hier wird die Funktion dann aufgerufen

['Ich', 'bin', 'ein', 'Test']

Funktionen, die mehrere Parameter bekommen sollen, können wir mithilfe der Funktion `partial` aus dem `functools`-Modul zu einargumentigen Funktionen machen. Beispiel ngram – wir wollen n=3 und sep='/' festlegen:

In [156]:
trigrams = partial(ngrams, n=3, sep='/')

`trigrams` kann nun mit einem Argument aufgerufen werden:

In [159]:
list(trigrams(["Ich", "bin", "ein", "Test"]))

['Ich/bin/ein', 'bin/ein/Test']

In [183]:
counters =pipeline(
        partial(corpus.forall, read_text, pattern="{filename}"),
        str.lower,
        tokenizer,
        partial(ngrams, n=2, sep=' '),
        Counter)

Das Ergebnis der pipeline-Funktion ist ein `map`-Objekt, das die gesamte Pipeline repräsentiert. Um zu einer Frequenztabelle zu kommen, machen wir einfach Standard-Pandas. Erst jetzt, durch das `list` (DataFrame mag keinen Iterator als `data`-Argument) wird die Pipeline ausgeführt!

In [185]:
data = pd.DataFrame(list(counters), index=corpus.metadata.index).fillna(0).T
data

basename,"Fontane,-Theodor_Der-Stechlin","Fontane,-Theodor_Effi-Briest","Fontane,-Theodor_Frau-Jenny-Treibel","Marlitt,-Eugenie_Das-Geheimnis-der-alten-Mamsell","Marlitt,-Eugenie_Das-Heideprinzesschen","Marlitt,-Eugenie_Die-Frau-mit-den-Karfunkelsteinen","Raabe,-Wilhelm_Die-Chronik-der-Sperlingsgasse","Raabe,-Wilhelm_Im-alten-Eisen","Raabe,-Wilhelm_Stopfkuchen"
1 1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1 2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1 4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1 april,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 auf,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1 bienenzüchter,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1 c,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 den,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1 er,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1 im,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Ausblick / TODO

Wenn das Modell so genehm ist, sind hier mögliche nächste Schritte:

1. Übertragung der Funktionen von oben in die Toolbox. Tests; ggf. noch Überarbeitungen / Modellierungsänderungen
2. Implementierung von tee (und ggf. consume) für `foreach`
3. Refactoring der existierenden Funktionen aus TMW, DARIAH-TM etc. nach dem modell, d.h. in kleine basale Funktionen ohne I/O à la ngram oder segmentize, sowie ggf. in Dateiparser bzw. -schreiber. Jeweils sinnvolle Definition der Parameter ermitteln.
3. Ggf. ergänzende Funktionen nach Bedarf. z.B. Featureextraktion aus DKPro-Wrapper-Format
4. Nachbau (ggf. nur als Demonstration) der ex. TMW / DARIAH-TM-Funktionen aus der Toolbox
5. Refactoring PyDelta/Next auf die Datenstrukturen hieraus:

    * FeatureExtractor als pipeline
    * CorpusDescriber als Mixin oder Subklasse von TableCorpus
    
6. Endnutzer-Tutorials: 

    1. basale Funktionen direkt benutzen
    2. mit Pipelines